<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/06_fine_tuning_llm/1_quantize_gemma_demo_w_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install libraries
!pip install -q transformers torch bitsandbytes accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.5 MB/s eta 0:00:00


In [2]:
import torch
import gc
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

In [3]:
# 2. Login to Hugging Face (Required for Gemma)
# Enter your token when prompted (get it from huggingface.co/settings/tokens)
login()

In [4]:
model_id = "google/gemma-2b"

def clean_memory():
    """Simple function to clear GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

In [5]:
print("\n" + "="*40)
print("PHASE 1: Standard Model (Float16)")
print("="*40)


PHASE 1: Standard Model (Float16)


In [6]:
# Load Standard Model
# We use float16 because that is the standard for modern LLMs (not float32)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
# Measure Size
mem_fp16 = model.get_memory_footprint() / 1024**3 # Convert to GB
print(f"Standard Memory Footprint: {mem_fp16:.2f} GB")

Standard Memory Footprint: 4.67 GB


In [8]:
# Clean up
del model
clean_memory()

print("\n" + "="*40)
print("PHASE 2: Quantized Model (4-bit)")
print("="*40)


PHASE 2: Quantized Model (4-bit)


In [9]:
# Configure 4-bit Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load Quantized Model
model_q = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# Measure Size
mem_4bit = model_q.get_memory_footprint() / 1024**3
print(f"Quantized Memory Footprint: {mem_4bit:.2f} GB")

# Show Comparison
print("-" * 30)
print(f"Memory Savings: {mem_fp16 / mem_4bit:.1f}x smaller")
print("-" * 30)

Quantized Memory Footprint: 1.90 GB
------------------------------
Memory Savings: 2.5x smaller
------------------------------
